### 对于Sqilte数据入库出现问题Debug
- author : yiran
- 错误描述：`peewee.InterfaceError: Error binding parameter 2 - probably unsupported type`
- 问题定位：通过错误信息的字面意思和搜索引擎查找得知，上面的error出现的问题可能是由于传入的数据类型出错
- 解决思路：
    1. 我们是将`BarData`传入到`Sqlite`数据库中，因此，传入数据类型出错的源头大概率出现在`BarData`上。
    2. `BarData`是通过使用`for row in imported_data.itertuples()`函数即对`imported_data`这个`DataFrame`逐行循环形成的。因此，我们需要对`imported_data`的各列的数据类型进行重新分析。
    3. 最有可能出现问题的是`datetime`列(个人经验)，我们可以先从这一列开始。我们可以分别把`dateime`列的数据类型分别设置为`datetime`数据类型和`str`数据类型，观察在这两种情况下,数据能否成功入库。如果都不成功，则尝试其它列。

In [1]:
import pandas as pd
import numpy as np
from vnpy.trader.constant import Exchange, Interval

#### 具体实现

为了验证上面提到的假设：`datetime`列的数据结构出现了问题，我设置了如下函数：
- `clean_rqdata_symbol_datetime`,该函数将`imported_data`中的`datetime`列数据类型设置为`datetime`数据类型
- `clean_rqdata_symbol_str`,该函数将`imported_data`中的`datetime`列数据类型设置为`str`数据类型
- `move_df_to_sql`，该函数将`DataFrame`中的数据入库到`Sqlite`数据库中

因此，具体的验证流程可以分解为：
1. 生成`imported_datetime`使用`clean_rqdata_symbol_datetime`
2. 生成`imoported_str`使用`clean_rqdata_symbol_str`
3. 使用`move_df_to_sql`，将`imported_datetime`和`imoported_str`分别入库，看是否有报错信息。

#### 数据说明
这里我们使用的example data 是rqdata的1分钟数据.


相应的example data和`.py`文件都可以在`Jupyter Notebook`所在目录找到


In [2]:

from Move_data_sql import clean_rqdata_symbol_datetime
from Move_data_sql import clean_rqdata_symbol_str
from Move_data_sql import move_df_to_sqlite

In [3]:
# 因为只是debug，我们只取20个bar的数据就可以
imported_data_datetime =  clean_rqdata_symbol_datetime('CU',Exchange.SHFE).iloc[:20]
imported_data_str =  clean_rqdata_symbol_str('CU',Exchange.SHFE).iloc[:20]

In [8]:
imported_data_datetime.head(2)

,datetime,close,dominant_id,high,low,open,open_interest,total_turnover,trading_date,volume,exchange,interval,symbol
0,2018-01-02 09:00:00,56740.0,CU1803,56860.0,56660.0,56760.0,220126.0,0.0,2018-01-02,3142.0,Exchange.SHFE,Interval.MINUTE,CU888
1,2018-01-02 09:01:00,56720.0,CU1803,56800.0,56710.0,56750.0,220062.0,0.0,2018-01-02,1068.0,Exchange.SHFE,Interval.MINUTE,CU888


In [9]:
imported_data_str.head(2)

,datetime,close,dominant_id,high,low,open,open_interest,total_turnover,trading_date,volume,exchange,interval,symbol
0,20180102 09:00:00,56740.0,CU1803,56860.0,56660.0,56760.0,220126.0,0.0,2018-01-02,3142.0,Exchange.SHFE,Interval.MINUTE,CU888
1,20180102 09:01:00,56720.0,CU1803,56800.0,56710.0,56750.0,220062.0,0.0,2018-01-02,1068.0,Exchange.SHFE,Interval.MINUTE,CU888


In [4]:
# 注意datetime对应的是datetime64数据类型
imported_data_datetime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
datetime          20 non-null datetime64[ns]
close             20 non-null float64
dominant_id       20 non-null object
high              20 non-null float64
low               20 non-null float64
open              20 non-null float64
open_interest     20 non-null float64
total_turnover    20 non-null float64
trading_date      20 non-null object
volume            20 non-null float64
exchange          20 non-null object
interval          20 non-null object
symbol            20 non-null object
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 2.2+ KB


In [5]:
# 注意datetime对应的是object数据类型
imported_data_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
datetime          20 non-null object
close             20 non-null float64
dominant_id       20 non-null object
high              20 non-null float64
low               20 non-null float64
open              20 non-null float64
open_interest     20 non-null float64
total_turnover    20 non-null float64
trading_date      20 non-null object
volume            20 non-null float64
exchange          20 non-null object
interval          20 non-null object
symbol            20 non-null object
dtypes: float64(7), object(6)
memory usage: 2.2+ KB


In [7]:
#将imported_data_str数据入库
move_df_to_sqlite(imported_data_str)

插入20 根bar 从 20180102 09:00:00 到 20180102 09:19:00


In [6]:
#将imported_data_datetime数据入库
move_df_to_sqlite(imported_data_datetime)

InterfaceError: Error binding parameter 2 - probably unsupported type.

#### 总结

经过上面的验证，我们发现出现问题的是`datetime`列的数据结构，在将数据入库到`Sqlite`数据库时，我们需要确保`datetime`列的数据类型是`str`.

另外，这里的`Debug`过程还是比较简单的，因为我们进行了一次验证就发现了问题。但在其它情况下，可能需要更多次的尝试和积极的思考。